<a href="https://colab.research.google.com/github/mhtarora39/ImgIdentifier/blob/master/Python_nlp_tensorflow2_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q tensorflow==2.0.0-alpha0
import tensorflow as tf

     |████████████████████████████████| 79.9MB 69.4MB/s 
     |████████████████████████████████| 61kB 25.6MB/s 
     |████████████████████████████████| 3.0MB 28.6MB/s 
     |████████████████████████████████| 419kB 58.7MB/s 


1) Load CSV
2) Remove stop word
3) Tokenize words
4) Convert in to word embedding vectors

In [0]:
import pandas as pd
import numpy as np

In [3]:

from google.colab import files
uploaded = files.upload()


Saving comments.csv to comments.csv


https://towardsdatascience.com/3-ways-to-load-csv-files-into-colab-7c14fcbdcb92  ** Link to use local files**

In [0]:
import io
df = pd.read_csv(io.BytesIO(uploaded["comments.csv"]))

In [5]:
df.head(5)

,approveDate,commentBody,commentID,commentSequence,commentTitle,commentType,createDate,depth,editorsSelection,parentID,parentUserDisplayName,permID,picURL,recommendations,recommendedFlag,replyCount,reportAbuseFlag,sharing,status,timespeople,trusted,updateDate,userDisplayName,userID,userLocation,userTitle,userURL,inReplyTo,articleID,sectionName,newDesk,articleWordCount,printPage,typeOfMaterial
0,1491245186,This project makes me happy to be a 30+ year T...,22022598,22022598,<br/>,comment,1491237056,1,False,0,NaN,22022598,https://graphics8.nytimes.com/images/apps/time...,2,NaN,0,NaN,0,approved,1,0,1491245186,Rob Gayle,46006296,"Riverside, CA",NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716,2,News
1,1491188619,Stunning photos and reportage. Infuriating tha...,22017350,22017350,NaN,comment,1491180489,1,False,0,NaN,22017350,https://graphics8.nytimes.com/images/apps/time...,1,NaN,0,NaN,0,approved,1,0,1491188619,Susan A.,29202761,<br/>,NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716,2,News
2,1491188617,Brilliant work from conception to execution. I...,22017334,22017334,<br/>,comment,1491179470,1,False,0,NaN,22017334,https://graphics8.nytimes.com/images/apps/time...,3,NaN,0,NaN,0,approved,1,0,1491188617,Meta,63944806,Raleigh NC,NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716,2,News
3,1491167820,NYT reporters should provide a contributor's l...,22015913,22015913,<br/>,comment,1491150196,1,False,0,NaN,22015913,https://graphics8.nytimes.com/images/apps/time...,7,NaN,2,NaN,0,approved,1,0,1491167820,Tom Wyrick,1266184,"Missouri, USA",NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716,2,News
4,1491167815,Could only have been done in print. Stunning.,22015466,22015466,<br/>,comment,1491147284,1,False,0,NaN,22015466,http://profile.ak.fbcdn.net/hprofile-ak-snc4/h...,5,NaN,0,NaN,0,approved,1,0,1491167815,Joe Sharkey,61121360,"Tucson, Arizona",NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716,2,News


``````````````````````````newDf[]

#filtered tweets
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
en_stops = list(set(stopwords.words('english')))

In [6]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
en_stops = list(set(stopwords.words('english')))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
df["commentBody_lower"] = df["commentBody"].str.lower()   

In [0]:
df["commentBody_removespacial"] = df["commentBody_lower"].str.replace('[^\w\s]','')

In [0]:
df["commentBody_clean"] = df["commentBody_removespacial"].apply(lambda x:' '.join(item for item in x.split() if item not in en_stops))

In [0]:
################Tokenizer step#####################

# Tokenizer step of keras assine token in sorted way 
# Lesser the number more frequent token is 

tok = tf.keras.preprocessing.text.Tokenizer(num_words=10000)
tok.fit_on_texts(list(df["commentBody_clean"]))

In [0]:
tf_train = tok.texts_to_sequences(list(df["commentBody_clean"]))

In [12]:
len(tf_train[0])

10

In [13]:
df["commentBody_clean"][0].count(' ')

11

In [0]:
max_len=0
index=-1
max_index=0
for item in tf_train:
  index=index+1
  if len(item) > max_len:
    max_len=len(item)
    max_index = index

In [15]:
print(max_index,max_len)

3461 150


In [0]:
tf_train = tf.keras.preprocessing.sequence.pad_sequences(tf_train,maxlen=200)

In [17]:
tf_train[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [18]:
df.head()

,approveDate,commentBody,commentID,commentSequence,commentTitle,commentType,createDate,depth,editorsSelection,parentID,parentUserDisplayName,permID,picURL,recommendations,recommendedFlag,replyCount,reportAbuseFlag,sharing,status,timespeople,trusted,updateDate,userDisplayName,userID,userLocation,userTitle,userURL,inReplyTo,articleID,sectionName,newDesk,articleWordCount,printPage,typeOfMaterial,commentBody_lower,commentBody_removespacial,commentBody_clean
0,1491245186,This project makes me happy to be a 30+ year T...,22022598,22022598,<br/>,comment,1491237056,1,False,0,NaN,22022598,https://graphics8.nytimes.com/images/apps/time...,2,NaN,0,NaN,0,approved,1,0,1491245186,Rob Gayle,46006296,"Riverside, CA",NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716,2,News,this project makes me happy to be a 30+ year t...,this project makes me happy to be a 30 year ti...,project makes happy 30 year times subscriber c...
1,1491188619,Stunning photos and reportage. Infuriating tha...,22017350,22017350,NaN,comment,1491180489,1,False,0,NaN,22017350,https://graphics8.nytimes.com/images/apps/time...,1,NaN,0,NaN,0,approved,1,0,1491188619,Susan A.,29202761,<br/>,NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716,2,News,stunning photos and reportage. infuriating tha...,stunning photos and reportage infuriating that...,stunning photos reportage infuriating trump ad...
2,1491188617,Brilliant work from conception to execution. I...,22017334,22017334,<br/>,comment,1491179470,1,False,0,NaN,22017334,https://graphics8.nytimes.com/images/apps/time...,3,NaN,0,NaN,0,approved,1,0,1491188617,Meta,63944806,Raleigh NC,NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716,2,News,brilliant work from conception to execution. i...,brilliant work from conception to execution iv...,brilliant work conception execution ive never ...
3,1491167820,NYT reporters should provide a contributor's l...,22015913,22015913,<br/>,comment,1491150196,1,False,0,NaN,22015913,https://graphics8.nytimes.com/images/apps/time...,7,NaN,2,NaN,0,approved,1,0,1491167820,Tom Wyrick,1266184,"Missouri, USA",NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716,2,News,nyt reporters should provide a contributor's l...,nyt reporters should provide a contributors li...,nyt reporters provide contributors link crowdf...
4,1491167815,Could only have been done in print. Stunning.,22015466,22015466,<br/>,comment,1491147284,1,False,0,NaN,22015466,http://profile.ak.fbcdn.net/hprofile-ak-snc4/h...,5,NaN,0,NaN,0,approved,1,0,1491167815,Joe Sharkey,61121360,"Tucson, Arizona",NaN,NaN,0,58def1347c459f24986d7c80,Unknown,Insider,716,2,News,could only have been done in print. stunning.,could only have been done in print stunning,could done print stunning


In [0]:
#Select the target to train the wordembeddings 
# ew require word_ids for compairing on tensor bord
classes = len(df["newDesk"].unique())

In [0]:
from sklearn import preprocessing
Y = df["newDesk"]
encoder = preprocessing.LabelEncoder()

In [21]:
#fit on catagory
encoder.fit(Y)

LabelEncoder()

In [0]:
# Transeform on token 
Y = encoder.transform(Y)

In [23]:
classes

10

In [0]:
Y = tf.keras.utils.to_categorical(y=Y,num_classes=classes)

In [25]:
Y[1]

array([0., 0., 0., 0., 1., 0., 0., 0., 0., 0.], dtype=float32)

In [26]:
classes

10

In [27]:
df["newDesk"].unique()

array(['Insider', 'OpEd', 'Editorial', 'Sports', 'Games', 'Culture',
       'Travel', 'Business', 'RealEstate', 'National'], dtype=object)

In [0]:
vocab_size = len(tok.word_index)+1

In [0]:
model = tf.keras.models.Sequential([ 
        tf.keras.layers.Embedding(input_length = 200,
                                 input_dim = vocab_size,
                                 output_dim = 50),
        
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(classes,activation=tf.nn.softmax)
                                  ])

In [0]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(tf_train,Y,test_size = 0.1,random_state=42)

In [0]:
model.compile(optimizer='adam',metrics=['accuracy'],loss='categorical_crossentropy')

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 200, 50)           1394250   
_________________________________________________________________
flatten (Flatten)            (None, 10000)             0         
_________________________________________________________________
dense (Dense)                (None, 10)                100010    
Total params: 1,494,260
Trainable params: 1,494,260
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.fit(x=np.array(x_train),y=np.array(y_train),epochs=5)

Epoch 1/5
5544/5544 [==============================] - 3s 537us/sample - loss: 1.6580 - accuracy: 0.4165
Epoch 2/5
5544/5544 [==============================] - 2s 441us/sample - loss: 1.0433 - accuracy: 0.6757
Epoch 3/5
5544/5544 [==============================] - 3s 458us/sample - loss: 0.5644 - accuracy: 0.8465
Epoch 4/5
5544/5544 [==============================] - 3s 509us/sample - loss: 0.3235 - accuracy: 0.9246
Epoch 5/5
5544/5544 [==============================] - 3s 506us/sample - loss: 0.2013 - accuracy: 0.9612


In [0]:
data = np.array(model.layers[0].get_weights())

In [35]:
data.shape

(1, 27885, 50)

In [0]:
data = data.reshape(data.shape[0]*data.shape[1],data.shape[2])
data = pd.DataFrame(data)

In [0]:
meta_data = pd.DataFrame.from_dict(list(tok.word_index))

In [0]:
meta_data.columns = ["word"]

In [39]:
meta_data.head()

,word
0,trump
1,people
2,would
3,one
4,like


In [0]:
meta_data["word"].to_csv("meta_data.csv",index=False,header=False)

In [0]:
files.download("meta_data.csv")

In [0]:
data[1:].to_csv("Embedding_weights_t.csv",sep='\t',index=False,header=False)

In [0]:
files.download("Embedding_weights_t.csv")